In [1]:
x=['من هو مؤلف الكتاب', 'اذكر إسم كاتب الكتاب', 'من الذى كتب الكتاب']
y=['ما هى مؤلفات الكاتب','مؤلفات الكاتب','كتب الكاتب','اذكر المؤلفات']
z=[' كم عدد الفصحات للكتاب','كم يمتلك صفحات ','ما هو عدد صفحات الكتاب']
link=['رابط قراءه الكتاب', 'اذكر رابط قراءه الكتاب', 'ما هو رابط الكتاب']
summary=[' ملخص كتاب', 'ما هو الملخص','اعطنى الملخص']
dic ={'اسم الكتاب':y,'الكاتب':x,'عدد صفحات الكتاب':z,'رابط قراءه الكتاب':link,'ملخص الكتاب':summary}

In [2]:
import nltk
from nltk.stem.isri import ISRIStemmer
stemmer = ISRIStemmer()
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC


In [3]:
#tokanize and stem uniqe words and save classes of questions asve all ov them in document 
words = []
classes = []
documents = []
ignore_words = ['?','؟',',','.']
for intent in dic:
    for pattern in dic[intent]:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent))
        if intent not in classes:
            classes.append(intent)

words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print (len(documents), "documents",documents)
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

16 documents [(['ما', 'هى', 'مؤلفات', 'الكاتب'], 'اسم الكتاب'), (['مؤلفات', 'الكاتب'], 'اسم الكتاب'), (['كتب', 'الكاتب'], 'اسم الكتاب'), (['اذكر', 'المؤلفات'], 'اسم الكتاب'), (['من', 'هو', 'مؤلف', 'الكتاب'], 'الكاتب'), (['اذكر', 'إسم', 'كاتب', 'الكتاب'], 'الكاتب'), (['من', 'الذى', 'كتب', 'الكتاب'], 'الكاتب'), (['كم', 'عدد', 'الفصحات', 'للكتاب'], 'عدد صفحات الكتاب'), (['كم', 'يمتلك', 'صفحات'], 'عدد صفحات الكتاب'), (['ما', 'هو', 'عدد', 'صفحات', 'الكتاب'], 'عدد صفحات الكتاب'), (['رابط', 'قراءه', 'الكتاب'], 'رابط قراءه الكتاب'), (['اذكر', 'رابط', 'قراءه', 'الكتاب'], 'رابط قراءه الكتاب'), (['ما', 'هو', 'رابط', 'الكتاب'], 'رابط قراءه الكتاب'), (['ملخص', 'كتاب'], 'ملخص الكتاب'), (['ما', 'هو', 'الملخص'], 'ملخص الكتاب'), (['اعطنى', 'الملخص'], 'ملخص الكتاب')]
5 classes ['اسم الكتاب', 'الكاتب', 'رابط قراءه الكتاب', 'عدد صفحات الكتاب', 'ملخص الكتاب']
18 unique stemmed words ['ؤلف', 'اسم', 'الذى', 'ذكر', 'ربط', 'صفح', 'عدد', 'عطنى', 'فصح', 'قرء', 'كتب', 'كم', 'لخص', 'ما', 'ملك', 'من', 'هو', 'هى']


In [4]:
#count vector fo every question of category 
training = []
output = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\PC\AppData\Local\Temp/ipykernel_14624/2411906996.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [5]:
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)

In [6]:
print("Train X :\n",train_x[:5])
print("\nTrain Y :\n",train_y[:5])

Train X :
 [[0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0]]

Train Y :
 [[0 0 1 0 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 1 0 0 0]]


In [7]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(50),solver='sgd',learning_rate_init=0.01,max_iter=1000)
mlp.fit(train_x, train_y)


MLPClassifier(hidden_layer_sizes=50, learning_rate_init=0.01, max_iter=1000,
              solver='sgd')

In [8]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1

    return(np.array(bag))
context = {}

ERROR_THRESHOLD = 0.50
def classify(sentence):
    results = mlp.predict([bow(sentence, words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence):
    results = classify(sentence)
    if results:
        while results:
            for i in doc:
                if i == results[0][0]:
                    print(i)

            return results

In [9]:
#part of speach tagging
Book_data=pd.read_csv("text (1).csv",encoding= 'utf-8-sig')
tokens_list=[]
tags_list=[]
columns=['عدد صفحات الكتاب','اسم الكتاب','الكاتب','القسم','دار النشر']
df_tranning=Book_data[columns]
q_W=['ما','ماذا','من','كم','اذكر']
pronouns=['التى','الذى','هم','هو','هى','تلك','هى' ,'هى']
frequ=['مؤلفات','مؤلف','كاتب','كتب','روايه','روايات','الفها','الكاتب','كاتب']
for i in q_W:
    tokens_list.append(i)
    tags_list.append('ادوات الأستفهام')
for i in pronouns:
    tokens_list.append(i)
    tags_list.append('الضمائر')
for i in frequ:
    tokens_list.append(i)
    tags_list.append('0')
for tag in df_tranning:
    for sentence in df_tranning[tag]:
        tokens = nltk.word_tokenize(str(sentence))
        for token in tokens:
            if token in q_W or token in pronouns or token in frequ  :
                continue 
            tokens_list.append(token)
            tags_list.append(tag)

In [10]:
dictionary={'word':tokens_list,'tag':tags_list}
dictionary

{'word': ['ما',
  'ماذا',
  'من',
  'كم',
  'اذكر',
  'التى',
  'الذى',
  'هم',
  'هو',
  'هى',
  'تلك',
  'هى',
  'هى',
  'مؤلفات',
  'مؤلف',
  'كاتب',
  'كتب',
  'روايه',
  'روايات',
  'الفها',
  'الكاتب',
  'كاتب',
  '10',
  '10',
  '10',
  '10',
  '10',
  '10',
  '10',
  '10',
  '10',
  '10',
  '10',
  '10',
  '10',
  '10',
  '10',
  '10',
  '10',
  '10',
  '10',
  '10',
  '339',
  '127',
  '28',
  '464',
  '95',
  '286',
  '28',
  '57',
  '125',
  '82',
  '69',
  '152',
  '16',
  '104',
  '216',
  '680',
  '86',
  '248',
  '58',
  '60',
  '58',
  '286',
  '56',
  '42',
  '50',
  '26',
  '401',
  '536',
  '40',
  '16',
  '1',
  '10',
  '13',
  '68',
  '7',
  '47',
  '10',
  '13',
  '7',
  '319',
  '844',
  '127',
  '282',
  '120',
  '10',
  '255',
  '175',
  '126',
  '64',
  '9',
  '44',
  '255',
  '403',
  '46',
  '61',
  '45',
  '46',
  '30',
  '24',
  '27',
  '58',
  '58',
  '48',
  '2',
  '10',
  '8',
  '16',
  '49',
  '10',
  '10',
  '9',
  '47',
  '239',
  '235',
  '10',
  '1

In [11]:
df = pd.DataFrame(dictionary)
df.to_csv('POS.csv',index=False,header=True,encoding='utf-8-sig')

In [12]:
df=pd.read_csv("POS.csv",encoding= 'utf-8-sig')


In [13]:
df=df.sample(frac=1)

In [14]:
#train model for categorize tokents tags
train, test = train_test_split(df, test_size = 0.4)
train_arr = []
test_arr = []
train_lbl = []
test_lbl = []

In [15]:
train_arr=train['word'].astype(str)
train_lbl=train['tag'].astype(str)
test_arr=test['word'].astype(str)
test_lbl=test['tag'].astype(str)

In [16]:
print("Train labels :\n",train_lbl[15:20])
print("\nTrain Words :\n",train_arr[15:20])

Train labels :
 63571     دار النشر
60359     دار النشر
43136        الكاتب
55305     دار النشر
20826    اسم الكتاب
Name: tag, dtype: object

Train Words :
 63571     الشيخ
60359       مصر
43136      سهيل
55305       مصر
20826    للحافظ
Name: word, dtype: object


In [17]:
vectorizer = CountVectorizer()
vectorizer.fit(train_arr)
train_mat = vectorizer.transform(train_arr)


In [18]:
tfidf = TfidfTransformer()
tfidf.fit(train_mat)
train_tfmat = tfidf.transform(train_mat)

In [19]:
test_mat = vectorizer.transform(test_arr)
test_tfmat = tfidf.transform(test_mat)
lsvm=LinearSVC()
lsvm.fit(train_tfmat,train_lbl)

LinearSVC()

In [20]:
y_pred_lsvm=lsvm.predict(test_tfmat)
test=['هى']
test_str = vectorizer.transform(test)
test_tfstr = tfidf.transform(test_str)
test_tfstr.shape
lsvm.predict(test_tfstr.toarray())[0]

'الضمائر'

In [21]:
from sklearn.metrics import  accuracy_score
from sklearn import metrics
print("accuracy:", metrics.accuracy_score(test_lbl, y_pred_lsvm))
print(test_lbl,'\n',y_pred_lsvm)

accuracy: 0.8939689833429064
17449          اسم الكتاب
35184              الكاتب
19157          اسم الكتاب
30374          اسم الكتاب
5274     عدد صفحات الكتاب
               ...       
386      عدد صفحات الكتاب
15461          اسم الكتاب
26854          اسم الكتاب
27064          اسم الكتاب
23291          اسم الكتاب
Name: tag, Length: 26115, dtype: object 
 ['اسم الكتاب' 'الكاتب' 'اسم الكتاب' ... 'اسم الكتاب' 'اسم الكتاب'
 'دار النشر']


In [22]:
# categorize question tokens
import sys
def categorize_Q_Tok(phrase):
    arr=phrase.split()
    y=[]
    token=[]
    for x in arr:
        x=[x]
        test_str = vectorizer.transform(x)
        test_tfstr = tfidf.transform(test_str)
        test_tfstr.shape
        token.append(x)
        y.append(lsvm.predict(test_tfstr.toarray())[0])
    return pd.DataFrame(list(zip(token,y)),columns=['token','entity_type'])


In [23]:
#extarct full index that used for quering needed data
def ExtractNames(Book_name,name,intent):
    if name in Book_name[intent] :
        return name
    for i in Book_name[intent]:
        c=0
        for j in name.split():
            if j in str(i).split():
                c+=1
        if c==len(name.split()) or c>len(str(i).split()):
            return i
    return 'NotFound'

In [29]:
def Answer(Book_data,intent,name,org):
    print(intent)
    temp = Book_data
    ans = []
    if name and intent=='اسم الكتاب':
        temp = temp.set_index('الكاتب')
        ExtractedName=ExtractNames(Book_data,name,'الكاتب')
        if ExtractedName!='NotFound':
            print(ExtractedName)
            ans.append(temp.loc[ExtractedName.strip(),intent])
        else:
            ans=ExtractedName
        return ans
    if org and intent =='الكاتب' :
        temp = temp.set_index('اسم الكتاب')
        ExtractedName=ExtractNames(Book_data,org,'اسم الكتاب')
        print(ExtractedName)
        if ExtractedName!='NotFound':
            ans.append(temp.loc[ExtractedName.strip(),intent])
        else:
            ans=ExtractedName
        return ans

In [25]:
def countNames(output):
    name=''
    org=''
    reg=''
    for i in range(len(output.loc[:,'entity_type'])):
        if output.loc[i,'entity_type'] == 'الكاتب' :
            name = name + output.loc[i,'token'][0] + ' '
    for i in range(len(output.loc[:,'entity_type'])):
        if output.loc[i,'entity_type'] == 'اسم الكتاب' :
            org = org + output.loc[i,'token'][0] + ' '
    return name , org 

In [32]:
def runing(phrase):
    name=''
    org=''
    intent = response(phrase)
    print(intent)
    name,org=countNames(categorize_Q_Tok(phrase))
    print(categorize_Q_Tok(phrase))
    ans=[]
    for i in range(len(intent)):
         ans.append(Answer(Book_data,intent[i][0],name,org))
    if ans !=[None]:
        print(ans)
        return ans
    else:
        print('من فضلك اعد السؤال بشكل اخر')
        return 'من فضلك اعد السؤال بشكل اخر'

In [44]:
import tkinter
from tkinter import *
class gui_selected3:
    def action(self):
        x=runing(self.e1.get('1.0',END))
    def __init__(self):
        self.root = Tk()
        self.root.title("Question Answering")
        self.root.geometry("400x300")
        self.QA = Label(self.root, text="Question ", font='Helvetica 10 bold')
        self.QA.pack()
        self.QA.place(x=50, y=20)
        self.e1 = tkinter.Text(self.root, width=20, height=2, bg='white', fg='black')
        self.e1.pack()
        self.e1.place(x=50, y=50)
        Subject = Label(self.root, text="Answer", font='Helvetica 10 bold')
        Subject.pack()
        Subject.place(x=50, y=100)
        self.e2 = tkinter.Text(self.root, width=30, height=8, bg='white', fg='black')
        self.e2.pack()
        self.e2.place(x=50, y=120)
        sbmitbtn = tkinter.Button(self.root, text="search", fg='white', bg="black", activebackground='white',
                                  activeforeground='black',
                                  height=1, font='Helvetica 10 bold',command=self.action)
        sbmitbtn.pack()
        sbmitbtn.place(x=260, y=50, height=35, width=70)
        self.root.mainloop()
gui_selected3()

[('اسم الكتاب', 1)]
      token entity_type
0      [ما]  اسم الكتاب
1      [هى]     الضمائر
2  [مؤلفات]           0
3  [الكاتب]           0
4    [محمد]      الكاتب
5   [عماره]  اسم الكتاب
اسم الكتاب
محمد 
محمد عمارة
[[الكاتب
محمد عمارة           نظرة جديدة إلى التراث
محمد عمارة           نظرة جديدة إلى التراث
محمد عمارة                تحديات لها تاريخ
محمد عمارة                تحديات لها تاريخ
محمد عمارة       تجديد الدنيا بتجديد الدين
                          ...             
محمد عمارة                    القرآن يتحدى
محمد عمارة    العلمانية بين الغرب والاسلام
محمد عمارة    العلمانية بين الغرب والاسلام
محمد عمارة             جمال الدين الأفغاني
محمد عمارة             جمال الدين الأفغاني
Name: اسم الكتاب, Length: 310, dtype: object]]
[('الكاتب', 1)]
     token      entity_type
0     [من]  ادوات الأستفهام
1     [هو]          الضمائر
2   [مؤلف]                0
3  [روايه]                0
4  [الكهف]       اسم الكتاب
الكاتب
الكهف 
رواية الكهف
[['الدكتور نبيل فاروق']]
